In [ ]:
''' VMP 2022-03-02: used in SI. 
Now uses one overall path '''

In [ ]:
# overall path to the project
path = "path/to/base"

In [ ]:
# check RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 54.8 gigabytes of available RAM



In [ ]:
# basic setup
%%capture
from google.colab import drive
drive.mount('/content/gdrive')
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar -xvzf spark-3.1.2-bin-hadoop3.2.tgz
! pip install -q findspark
! pip install pyspark

In [ ]:
# new try 
import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [ ]:
# import stuff
import findspark
findspark.init()
import sys
import pandas as pd
sys.path.insert(0, f'{path}/CODE/Database')
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col
from MAG import MicrosoftAcademicGraph
from MAGspark1 import get_mag_with_node_connection 
import datetime, time 
mag, spark = get_mag_with_node_connection()

In [ ]:
''' dataset for collaboration network '''

' dataset for collaboration network '

In [ ]:
## get the 10 pct ##

In [ ]:
# sample 10% of unique papers 
percentage = mag.getDataframe('PaperAuthorAffiliationsAttributesAll') \
  .select('PaperId') \
  .filter(col('Date') < datetime.date(2021, 7, 1)) \
  .distinct() \
  .sample(0.1, seed = 41) \

In [ ]:
# subset of the data for collaboration.ipynb
selection = mag.getDataframe('PaperAuthorAffiliationsAttributesAll') \
  .select('PaperId', 'AuthorId', 'Date', 'ScientificAge', 'Gender', 'NormalizedName') \
  .join(percentage, ["PaperId"], "inner") \
  .distinct() \

In [ ]:
# collaboration
selection.select('PaperId', 'AuthorId', 'Date') \
  .distinct() \
  .toPandas() \
  .to_csv(f"{path}/DATA/collaboration/network_SI/preprocessing/paaaa_collaboration.csv", index = False)

In [ ]:
# for main 
selection.select('PaperId', 'AuthorId', 'Date', 'ScientificAge') \
  .distinct() \
  .toPandas() \
  .to_csv(f"{path}/DATA/collaboration/network_SI/preprocessing/paaaa_main.csv", index = False)

In [ ]:
# for something else 
selection.select('AuthorId', 'Gender') \
  .distinct() \
  .toPandas() \
  .to_csv(f"{path}/DATA/collaboration/network_SI/preprocessing/AuthorGenderAll.csv", index = False)

In [ ]:
# main field of study ([2010-2021-06])
paaaaFoS = selection.select('PaperId', 'AuthorId', 'Date', 'NormalizedName') \
  .distinct() \
  .toPandas()

In [ ]:
# legit fields 
fields = ["biology", "chemistry", "computer science", "engineering", "environmental science", "geography", "geology", "materials science", "mathematics", "physics"]

In [ ]:
# only in legit fields
paaaaFoSmain = paaaaFoS[paaaaFoS['NormalizedName'].isin(fields)]

In [ ]:
''' changed it (most recent rather than mode) to make it faster '''

' trying to make it a bit faster '

In [ ]:
# FoS (takes forever)
paaaaFoSunique = paaaaFoSmain.sort_values('Date').groupby(['AuthorId']).first().reset_index() # just most recent instead. 
paaaaFoSunique = paaaaFoSunique[["AuthorId", "NormalizedName"]]
paaaaFoSunique = paaaaFoSunique.sample(frac=1).drop_duplicates(subset='AuthorId').reset_index(drop=True) # actually not necessary now.. 
paaaaFoSunique.to_csv(f"{path}/DATA/collaboration/network_SI/preprocessing/paaaaFoSunique.csv", index = False)

In [ ]:
# merge inner with other Author information
AuthorGenderRepo = pd.read_csv(f"{path}/DATA/collaboration/network_SI/preprocessing/AuthorGenderAll.csv")
AuthorGenderFoSRepo = AuthorGenderRepo.merge(paaaaFoSunique, on = "AuthorId", how = "inner")
AuthorGenderFoSRepo.to_csv(f"{path}/DATA/collaboration/network_SI/preprocessing/AuthorCountryGenderFoSAll.csv", index = False)